In [1]:
import pandas as pd

#load in the data
df1 = pd.read_csv('gemeente_data.csv', delimiter=";")
print(df1.head())

df2 = pd.read_csv('tussen_gemeente_verhuisde_personen.csv', delimiter=";")
df2.head()


   ID      RegioS      Code_1  \
0   0  GM1680      GM1680       
1   1  GM0358      GM0358       
2   2  GM0197      GM0197       
3   3  GM0059      GM0059       
4   4  GM0482      GM0482       

                                              Naam_2  \
0  Aa en Hunze                                   ...   
1  Aalsmeer                                      ...   
2  Aalten                                        ...   
3  Achtkarspelen                                 ...   
4  Alblasserdam                                  ...   

                                     SorteringNaam_3  Inwonertal_54  
0  Aa en Hunze                                   ...          25724  
1  Aalsmeer                                      ...          33063  
2  Aalten                                        ...          27244  
3  Achtkarspelen                                 ...          28149  
4  Alblasserdam                                  ...          20356  


,ID,RegioVanVestiging,RegioVanVertrek,Perioden,TussenGemeentenVerhuisdePersonen_1
0,32705,PV20,PV20,2021JJ00,13109.0
1,32718,PV20,PV21,2021JJ00,2991.0
2,32731,PV20,PV22,2021JJ00,5051.0
3,32744,PV20,PV23,2021JJ00,1520.0
4,32757,PV20,PV24,2021JJ00,500.0


In [6]:
#cleanup the data

# Check the data types of GM codes in both dataframes
print("Data type of df1['RegioS']:", df1['RegioS'].dtype)
print("Data type of df2['RegioVanVestiging']:", df2['RegioVanVestiging'].dtype)
print("Data type of df2['RegioVanVertrek']:", df2['RegioVanVertrek'].dtype)

# Check for leading/trailing whitespace in GM codes
print("Leading/trailing whitespace in df1['RegioS']:", df1['RegioS'].str.contains(r'^\s+|\s+$').any())
print("Leading/trailing whitespace in df2['RegioVanVestiging']:", df2['RegioVanVestiging'].str.contains(r'^\s+|\s+$').any())
print("Leading/trailing whitespace in df2['RegioVanVertrek']:", df2['RegioVanVertrek'].str.contains(r'^\s+|\s+$').any())

# Display unique values for manual inspection
print("Sample of unique values in df1['RegioS']:", df1['RegioS'].unique()[:5])
print("Sample of unique values in df2['RegioVanVestiging']:", df2['RegioVanVestiging'].unique()[:5])
print("Sample of unique values in df2['RegioVanVertrek']:", df2['RegioVanVertrek'].unique()[:5])

# Strip leading and trailing whitespaces and ensure all values are strings
df1['RegioS'] = df1['RegioS'].str.strip().astype(str)
df2['RegioVanVestiging'] = df2['RegioVanVestiging'].str.strip().astype(str)
df2['RegioVanVertrek'] = df2['RegioVanVertrek'].str.strip().astype(str)

# Verify the cleaning
print("Data type of df1['RegioS'] after cleaning:", df1['RegioS'].dtype)
print("Data type of df2['RegioVanVestiging'] after cleaning:", df2['RegioVanVestiging'].dtype)
print("Data type of df2['RegioVanVertrek'] after cleaning:", df2['RegioVanVertrek'].dtype)

# Display sample cleaned values
print("Sample values in df1['RegioS'] after cleaning:", df1['RegioS'].unique()[:5])
print("Sample values in df2['RegioVanVestiging'] after cleaning:", df2['RegioVanVestiging'].unique()[:5])
print("Sample values in df2['RegioVanVertrek'] after cleaning:", df2['RegioVanVertrek'].unique()[:5])



Data type of df1['RegioS']: object
Data type of df2['RegioVanVestiging']: object
Data type of df2['RegioVanVertrek']: object
Leading/trailing whitespace in df1['RegioS']: True
Leading/trailing whitespace in df2['RegioVanVestiging']: True
Leading/trailing whitespace in df2['RegioVanVertrek']: True
Sample of unique values in df1['RegioS']: ['GM1680    ' 'GM0358    ' 'GM0197    ' 'GM0059    ' 'GM0482    ']
Sample of unique values in df2['RegioVanVestiging']: ['PV20  ' 'PV21  ' 'PV22  ' 'PV23  ' 'PV24  ']
Sample of unique values in df2['RegioVanVertrek']: ['PV20  ' 'PV21  ' 'PV22  ' 'PV23  ' 'PV24  ']
Data type of df1['RegioS'] after cleaning: object
Data type of df2['RegioVanVestiging'] after cleaning: object
Data type of df2['RegioVanVertrek'] after cleaning: object
Sample values in df1['RegioS'] after cleaning: ['GM1680' 'GM0358' 'GM0197' 'GM0059' 'GM0482']
Sample values in df2['RegioVanVestiging'] after cleaning: ['PV20' 'PV21' 'PV22' 'PV23' 'PV24']
Sample values in df2['RegioVanVertre

In [2]:
df2_filtered = df2[df2['RegioVanVestiging'].str.startswith('GM') & df2['RegioVanVertrek'].str.startswith('GM')]

df_merged_filtered = df2_filtered.merge(df1[['RegioS', 'SorteringNaam_3']], 
                                        left_on='RegioVanVestiging', right_on='RegioS', 
                                        how='left').rename(columns={'SorteringNaam_3': 'Naam_RegioVanVestiging'})

df_merged_filtered = df_merged_filtered.merge(df1[['RegioS', 'SorteringNaam_3']], 
                                              left_on='RegioVanVertrek', right_on='RegioS', 
                                              how='left').rename(columns={'SorteringNaam_3': 'Naam_RegioVanVertrek'})

# Drop unnecessary columns
df_merged_filtered = df_merged_filtered.drop(columns=['RegioS_x', 'RegioS_y'])

df_merged_filtered = df_merged_filtered.rename({
    "RegioVanVestiging" : "RegionToID",
    "RegioVanVertrek" : "RegionFromID",
    "Perioden" : "Timeframe",
    "TussenGemeentenVerhuisdePersonen_1" : "AmountOfPeople",
    "Naam_RegioVanVestiging" : "RegionToName",
    "Naam_RegioVanVertrek" : "RegionFromName"
}, axis = 1).dropna()


df_merged_filtered.to_csv('merged_cleaned.csv', index=False)

# Display the merged dataframe
df_merged_filtered.head()


,ID,RegionToID,RegionFromID,Timeframe,AmountOfPeople,RegionToName,RegionFromName
